In [1]:
import pandas as pd
import glob
import time
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import metrics

In [26]:
pd.set_option('display.max_rows', 100)

最終日にアンサンブルつくろうと思って、LBよかった3ファイル混ぜてみた。  
混ぜ方めっちゃ迷ったけど、  
- 0507が寿命90以下は一番いいはず(謎の自信)
- ただし、寿命100付近の切れ目の部分はちょっと自信なさげ
- 0414と0428は、特にpublicではいいスコア踏んでるので、うまく使いたい
- そこで、0507で0~90はそのまま、90以上は3ファイルの平均とした
- LB13.2とかだったので、これを最終ファイルとした

In [6]:
df_0414 = pd.read_csv('C0003_14_04_2019.csv')
df_0428 = pd.read_csv('C0003_28_04_2019.csv')
df_0507 = pd.read_csv('C0003_07_05_2019.csv')

In [12]:
df = pd.concat([df_0414, df_0428, df_0507], axis=1)
df.columns = ['Predicted RUL 0414', 'Predicted RUL 0428', 'Predicted RUL 0507']

In [13]:
df.corr()

,Predicted RUL 0414,Predicted RUL 0428,Predicted RUL 0507
Predicted RUL 0414,1.000000,0.9908,0.965128
Predicted RUL 0428,0.990800,1.0000,0.966600
Predicted RUL 0507,0.965128,0.9666,1.000000


In [16]:
df[df['Predicted RUL 0507'] > 100].corr()

,Predicted RUL 0414,Predicted RUL 0428,Predicted RUL 0507
Predicted RUL 0414,1.000000,0.958627,0.876850
Predicted RUL 0428,0.958627,1.000000,0.886257
Predicted RUL 0507,0.876850,0.886257,1.000000


In [17]:
df[df['Predicted RUL 0507'] < 100].corr()

,Predicted RUL 0414,Predicted RUL 0428,Predicted RUL 0507
Predicted RUL 0414,1.000000,0.984901,0.933851
Predicted RUL 0428,0.984901,1.000000,0.918172
Predicted RUL 0507,0.933851,0.918172,1.000000


cvは0507が圧倒的にいいはず。
100以下は0507を信頼して、100以上は3ファイルを平均する

In [23]:
df['Predicted RUL'] = df['Predicted RUL 0507']
df.loc[df['Predicted RUL 0507'] > 90, 'Predicted RUL'] = df.mean(axis=1)

In [27]:
df

,Predicted RUL 0414,Predicted RUL 0428,Predicted RUL 0507,Predicted RUL
0,131.792078,130.711272,123.383454,127.317564
1,40.393972,51.240830,31.510975,31.510975
2,125.668150,117.179415,133.072309,127.248046
3,159.692731,156.324702,169.027735,163.518226
4,5.683622,6.124391,8.623388,8.623388
5,113.382292,120.064437,116.261355,116.492360
6,17.932336,15.637728,10.553555,10.553555
7,124.364778,144.141969,144.487082,139.370228
8,103.269002,107.805987,64.929652,64.929652
9,42.254540,50.368203,35.270457,35.270457


In [29]:
df[['Predicted RUL']].to_csv('C0003_09_05_2019.csv', index=False)